In [1]:
import pandas as pd

In [2]:
df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
list_url

总数量：577


['https://www.dormanproducts.com/p-106667-626-609.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-104629-603-771.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-104640-603-758.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-103807-904-7703.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-104641-603-768.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-104969-904-7718.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-86574-603-466.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-139788-904-7363.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-83386-603-184.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-84955-902-784.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-83425-603-169.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-51561-924-5205.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-83385-603-061.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-7849-603-001.aspx?origin=keywor

In [3]:
df = pd.DataFrame(columns=['Url',
                           'OE_Numbers',
                           'vehicle_url',
                           'Src',
                           'status'])

df

,Url,OE_Numbers,vehicle_url,Src,status


In [4]:
import requests

In [5]:
i = 0

resp = requests.get(list_url[i]).text

resp

'<head><title>ShieldSquare Captcha</title><script type="text/javascript">\n\twindow.SSJSInternal = 34673;\n\n    var __uzdbm_1 = "c7e69871-4125-461c-841a-6a75879f8915";\n    var __uzdbm_2 = "ZGQyZjdiYmEtYnUydi00YzhhLThlOWItNmEwOGYzNWY4Yjc4JDEyMC4zMi4xMjUuMTAy";\n\n\t(function(w, d, e, u, c, g, a, b){\n\t\tw["SSJSConnectorObj"] = w["SSJSConnectorObj"] || {ss_cid : c, domain_info: g};\n\t\ta = d.createElement(e);\n\t\ta.async = true;\n\t\ta.src = u;\n\t\tb = d.getElementsByTagName(e)[0];\n\t\tb.parentNode.insertBefore(a, b);\n\t})(window,document,"script","https://cdn.perfdrive.com/aperture/aperture.js","b8c3","auto");\n</script>\n</head><div>\n\n<style>\n    h1 { font-size: 36px; margin: 10px 0;}\n    img { vertical-align: middle; }\n    .container { font-size: 15px; margin: 20px; line-height: 1.5; color: #333; background-color: #fff; font-family: "open sans", arial, sans-serif;}\n    .row { margin: 10px; height: auto; text-align: left; }\n    .footer { font-size: 14px; background-color

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(resp, 'lxml')

print(soup.prettify())

<html>
 <head>
  <title>
   ShieldSquare Captcha
  </title>
  <script type="text/javascript">
   window.SSJSInternal = 34673;

    var __uzdbm_1 = "c7e69871-4125-461c-841a-6a75879f8915";
    var __uzdbm_2 = "ZGQyZjdiYmEtYnUydi00YzhhLThlOWItNmEwOGYzNWY4Yjc4JDEyMC4zMi4xMjUuMTAy";

	(function(w, d, e, u, c, g, a, b){
		w["SSJSConnectorObj"] = w["SSJSConnectorObj"] || {ss_cid : c, domain_info: g};
		a = d.createElement(e);
		a.async = true;
		a.src = u;
		b = d.getElementsByTagName(e)[0];
		b.parentNode.insertBefore(a, b);
	})(window,document,"script","https://cdn.perfdrive.com/aperture/aperture.js","b8c3","auto");
  </script>
 </head>
 <body>
  <div>
   <style>
    h1 { font-size: 36px; margin: 10px 0;}
    img { vertical-align: middle; }
    .container { font-size: 15px; margin: 20px; line-height: 1.5; color: #333; background-color: #fff; font-family: "open sans", arial, sans-serif;}
    .row { margin: 10px; height: auto; text-align: left; }
    .footer { font-size: 14px; background-colo

In [8]:
with open('./part.html', 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [9]:
from lxml import etree

In [10]:
html = etree.HTML(str(soup))

oe = ';'.join(html.xpath('//section[@id="productOE"]/descendant::*/th[@scope="row"]/text()'))

oe

''

In [11]:
if len(html.xpath('//input[@id="hidfDetailApp"]/@value')) == 0:
    vehicle_url = ''
else:
    vehicle_url = 'https://www.dormanproducts.com/' + html.xpath('//input[@id="hidfDetailApp"]/@value')[0]

vehicle_url

''

In [12]:
src = html.xpath('//img[@id="imgProductMain"]/@src')[0]

src

IndexError: list index out of range

In [ ]:
df_temp = pd.DataFrame([{'Url': list_url[i],
                         'OE_Numbers': oe,
                         'vehicle_url': vehicle_url,
                         'Src': src,
                         'status': 'ok'}])

df_temp

In [ ]:
list_row = html.xpath('//section[@id="productSpec"]/descendant::tr')
for row in list_row:
    df_temp[row.xpath('./th')[0].text.strip()[:-1].lower()] = row.xpath('./td')[0].text
    
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df